In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Exploring observed patterns in biomass relationships between seascapes
We try to explore the patterns in the outcome of the causal association network between species. What features might best explain the existence or absence of a causal dependency between neighbouring seascapes at the 0, 1 and 2 year lag.

- First we create a new dataset listing the following combination of survey and species specific life-history features:  
    (1) pairwise combinations of neighbouring seascapes  
    (2) species  
    (3) family  
    (4) year  
    (5) absolute difference in small juvenile biomass  
    (5) absolute difference in large juvenile biomass
    (6) absolute difference in small adult biomass  
    (7) absolute difference in large adult biomass
    (8) migratory (yes/no) trait  
    (9) habitat trait  
    (10) feeding mode  
    (11) maximum length trait  
    (12) body type   
    (13) spawning-type  
      
    then the label feature a Random Forest model is trained to predict is either of the following three:  
    (14) 0 year lag (yes/no)/ 1 year lag (yes/no)/ 2 year lag (yes/no)  
  
    And shapley values are used to extract the variable importance from the model in the prediction of the label:  
    (https://christophm.github.io/interpretable-ml-book/shapley.html)  
      
    *trait database of marine fish:  
    https://doi.pangaea.de/10.1594/PANGAEA.900866  
    *fishbase:   
    https://www.fishbase.de/  
    

In [15]:
def make_identifier(df):
    str_id = df.apply(lambda x: '_'.join(map(str, x)), axis=1)
    return pd.factorize(str_id)[0]

import pandas as pd
import numpy as np

user = "/Users/markrademaker/Projects/Sea-scape-analysis/"

### Dataset with separate Juv and Adult Biomass column
species_list = ["Gadus_morhua",
               "Pleuronectes_platessa",
               "Eutrigla_gurnardus",
               "Sprattus_sprattus",
               "Limanda_limanda",
               "Clupea_harengus",
               "Amblyraja_radiata",
               "Merlangius_merlangus",
               "Callionymus_lyra"]

life_stage = ["Juvenile","Adult"]

maturation_length=[68.3,#Gadus morhua
                   30.8,#Pleuronectes platessa
                   18.8,#Eutrigla gurnardus
                   10.1,#Sprattus sprattus
                   21.4,#Limanda limanda
                   16.7,#Clupea harengus
                   38,#Amblyraja radiata (30m,32f,44m,46f (Mcculley et al., 2012))
                   27.8,#Merlangius merlangus
                   10.3#Callionymus lyra (8m, 11fm (King et al.,1994), 12 (Jost,1944))
                  ]

spec_dataframe_list =[]
for i in range(len(species_list)):
    spec = species_list[i]
    
    df = pd.read_csv(user+"cpue_length_hour/Filtered_{}.csv".format(spec))
    print(df["Species"][1])
    df = df[df["Year"] > 1977]
    
    #subset data Quarter 1
    #df = df[df["Quarter"]==1]
    df["Biomass_kg"] = df["Total_wgt"]/1000
    #df["Seascapenr"] = df["id"]
    
    #df['LngtClass_cm'].replace(to_replace=0, value=np.nan, inplace=True)
    
    df["ID"] = make_identifier(df[["Ship","Gear","HaulNo","HaulDur","ShootLat","ShootLong"]])
    
    #print(df["ID"].unique())
    #print(np.sort(df["LngtClass_cm"].unique()))
    
    #df_agg = df.groupby("Biomass_kg")["ID","Year","Seascapenr","Species","Ship","DayNight"].agg("sum")
    df_agg = df.groupby(["ID","Year","Seascapenr","Species","Ship","DayNight","LngtClass_cm"])["Biomass_kg"].sum().reset_index()
    #print(len(df_agg))
    
    #f_agg_1977= df_agg[df_agg["Year"]==1977]
    #print(df_agg["Seascapenr"].unique())
    
    ##Zero catch data
    df_zero = df_agg[df_agg['LngtClass_cm']==0]
    #print("length zero = "+str(len(df_zero)))
    
    ##Positive catch data
    df_agg = df_agg[df_agg["LngtClass_cm"] > 0]
    #print("max length = "+str(max(df_agg["LngtClass_cm"])))
    #print("min length = "+str(min(df_agg["LngtClass_cm"])))
    
        
    ##Juvenile data
    df_juv = df_agg[df_agg['LngtClass_cm'] < maturation_length[i]]
    #print("length juv data = "+str(len(df_juv)))
    ##Split into lower and upper quantile of length
    df_juv['Length_quantile'] =  pd.qcut(df_juv["LngtClass_cm"],2,labels=False)
    
    #lower q
    df_juv_lq = df_juv[df_juv['Length_quantile']==0]
    df_juv_lq["LQL Juvenile Biomass"] = df_juv_lq["Biomass_kg"]
    #print("length lq = "+str(len(df_juv_lq)))
   
    #upper q
    df_juv_uq = df_juv[df_juv['Length_quantile']==1]
    df_juv_uq["UQL Juvenile Biomass"] = df_juv_uq["Biomass_kg"]
    #print("length uq = "+str(len(df_juv_uq)))
    
    ##Adult data
    df_ad = df_agg[df_agg['LngtClass_cm'] > maturation_length[i]]
    #print("length adult data = "+str(len(df_ad)))
    ##Split into lower and upper quantile of length
    df_ad['Length_quantile'] =  pd.qcut(df_ad["LngtClass_cm"],2,labels=False)
    
    #lower q
    df_ad_lq = df_ad[df_ad['Length_quantile']==0]
    df_ad_lq["LQL Adult Biomass"] = df_ad_lq["Biomass_kg"]
    #print("length lq = "+str(len(df_ad_lq)))
    
    #upper q
    df_ad_uq = df_ad[df_ad['Length_quantile']==1]
    df_ad_uq["UQL Adult Biomass"] = df_ad_uq["Biomass_kg"]
    #print(df_ad_uq)
    #print("length uq = "+str(len(df_ad_uq)))
    
    
    
    ##Bring into single dataframe
    frames=[df_juv_lq,df_juv_uq,df_ad_lq,df_ad_uq,df_zero]
    df_new = pd.concat(frames)
    df_new = df_new.groupby(['ID','Year','Species','Seascapenr'])["LQL Juvenile Biomass","UQL Juvenile Biomass",
                                                         "LQL Adult Biomass","UQL Adult Biomass","Biomass_kg"].sum().reset_index()
    df_new["Juvenile Biomass"] = df_new["LQL Juvenile Biomass"] + df_new["UQL Juvenile Biomass"]
    df_new["Adult Biomass"] = df_new["LQL Adult Biomass"] + df_new["UQL Adult Biomass"]
    #df_new.to_csv(user+"cpue_length_hour/{}_adult_juvenile_biomass.csv".format(spec))
    
    df_new = df_new.groupby(['Species','Year','Seascapenr'])["LQL Juvenile Biomass","UQL Juvenile Biomass",
                                                         "LQL Adult Biomass","UQL Adult Biomass","Adult Biomass","Juvenile Biomass"].mean().reset_index()
    
    df_new.to_csv(user+"cpue_length_hour/{}_adult_juvenile_biomass.csv".format(spec))
    
    spec_dataframe_list.append(df_new)

df_big = pd.concat(spec_dataframe_list)
df_big = df_big.reset_index(drop=True)
print(df_big.tail())

df_big.to_csv(user+"RF/species_juv_ad_biomass.csv".format(spec))
    
#df_new.head()

Gadus morhua


/Users/markrademaker/opt/anaconda3/envs/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/markrademaker/opt/anaconda3/envs/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/markrademaker/opt/anaconda3/envs/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

Pleuronectes platessa
Eutrigla gurnardus
Sprattus sprattus
Limanda limanda
Clupea harengus
Amblyraja radiata
Merlangius merlangus
Callionymus lyra
               Species  Year  Seascapenr  LQL Juvenile Biomass  \
3775  Callionymus lyra  2019         6.0              0.002295   
3776  Callionymus lyra  2019         7.0              0.000000   
3777  Callionymus lyra  2019         8.0              0.000148   
3778  Callionymus lyra  2019         9.0              0.000000   
3779  Callionymus lyra  2019        10.0              0.007489   

      UQL Juvenile Biomass  LQL Adult Biomass  UQL Adult Biomass  \
3775              0.000000           0.005079           0.000000   
3776              0.000000           0.000000           0.010342   
3777              0.000000           0.018156           0.011586   
3778              0.000000           0.025934           0.139563   
3779              0.004483           0.029029           0.069311   

      Adult Biomass  Juvenile Biomass  
3775   

#### Now we link this table to the traits table we created before

In [16]:
import pandas as pd
import numpy as np
import xlrd

user = "/Users/markrademaker/Projects/Sea-scape-analysis/"



trait_table = pd.read_excel(user+'RF/Seascape_trait_causality_table_2.xlsx',sheet_name="Seascape_species")
causality_table = pd.read_excel(user+'RF/Seascape_trait_causality_table_2.xlsx',sheet_name="Seascape comb. lag 1-2")
causality_table.columns
causality_table_zero = causality_table[["Seascape combination","Neighbouring","Species","Causal lag 0"]]
trait_table.head()
causality_table_zero.head()



Index(['Seascape combination', 'Neighbouring', 'Species', 'Causal lag 0',
       'value', 'Causal lag 1', 'value.1', 'direction', 'Causal lag 2',
       'value.2', 'direction.1'],
      dtype='object')

,Species,Family,Seasonal migration,Fecundity,Spawning type,Age at Maturity,Maximum age,Habitat,Feeding mode,Body type,Maximum length,Length at maturity
0,Gadus morhua,Gadidae,yes,4.836395e+06,non-guarder,4.858807,25.000000,benthopelagic,generalist,fusiform,116.258333,68.3
1,Pleuronectes platessa,Pleuronectidae,yes,5.520000e+05,non-guarder,4.500000,30.857143,demersal,benthivorous,flat,100.000000,30.8
2,Eutrigla gurnardus,Triglidae,yes,2.449490e+05,non-guarder,2.775000,2.775000,demersal,generalist,elongated,36.000000,18.8
3,Sprattus sprattus,Clupeidae,yes,1.463000e+04,non-guarder,2.000000,6.000000,pelagic,planktivorous,fusiform,14.440000,10.1
4,Limanda limanda,Pleuronectidae,yes,2.224286e+05,non-guarder,2.985714,12.142857,demersal,benthivorous,flat,43.428571,21.4


,Seascape combination,Neighbouring,Species,Causal lag 0
0,1-2,yes,Gadus morhua,no
1,1-3,yes,Gadus morhua,no
2,1-4,no,Gadus morhua,no
3,1-5,no,Gadus morhua,no
4,1-6,no,Gadus morhua,no


For each species, year and seascape combination we calculate the difference in each of the four biomass categories between (LQL Juv, UQL Juv, LQL Adult, UQL Adult, i.e. small/large juveniles/adults)

In [17]:
import pandas as pd
import numpy as np
import xlrd

user = "/Users/markrademaker/Projects/Sea-scape-analysis/"

#necessary tables
df_big=pd.read_csv(user+"RF/species_juv_ad_biomass.csv")
years = df_big["Year"].unique()
trait_table = pd.read_excel(user+'RF/Seascape_trait_causality_table_2.xlsx',sheet_name="Seascape_species")
causality_table = pd.read_excel(user+'RF/Seascape_trait_causality_table_2.xlsx',sheet_name="Seascape comb. lag 1-2")

#lag 0,1,2
causality_table = causality_table[["Seascape combination","Neighbouring","Species","Causal lag 0","Causal lag 1","Causal lag 2"]]
final = pd.concat([causality_table.assign(year=y) for y in years], ignore_index=True)
final.rename(columns = {"year": "Year"},  inplace = True)
final.to_csv(user+"RF/lag_combined_data_with_year.csv")



#Species
species_list = causality_table["Species"].unique()
#Seascape combinations
seascape_combi_list = causality_table["Seascape combination"].unique()
#Years
years = df_big["Year"].unique()
print(years)

diff_biomass = []

for i in range(len(species_list)):
    spec = species_list[i]
    print (spec)

    spec_df = df_big[df_big["Species"]==spec]
    
    for year in years:
        print(year)
        spec_year_df = spec_df[spec_df["Year"]==year]
        
        for combi in seascape_combi_list:
            s1 = int(combi[0])
            s2 = int(combi[2:])
            
            s1_df = spec_year_df[spec_year_df["Seascapenr"]==s1]
            s2_df = spec_year_df[spec_year_df["Seascapenr"]==s2]
            
            diff_lql_juv = s1_df["LQL Juvenile Biomass"].iloc[0]-s2_df["LQL Juvenile Biomass"].iloc[0] 
            print(s1_df["LQL Juvenile Biomass"].iloc[0],s2_df["LQL Juvenile Biomass"].iloc[0])
            print(s1_df["LQL Juvenile Biomass"].iloc[0]-s2_df["LQL Juvenile Biomass"].iloc[0])
           
            #print(diff_lql_juv)
            diff_uql_juv = s1_df["UQL Juvenile Biomass"].iloc[0]-s2_df["UQL Juvenile Biomass"].iloc[0]
            print(s1_df["UQL Juvenile Biomass"].iloc[0],s2_df["UQL Juvenile Biomass"].iloc[0])
            print(s1_df["UQL Juvenile Biomass"].iloc[0]-s2_df["UQL Juvenile Biomass"].iloc[0])
           
            diff_lql_ad = s1_df["LQL Adult Biomass"].iloc[0]-s2_df["LQL Adult Biomass"].iloc[0]
            diff_uql_ad = s1_df["UQL Adult Biomass"].iloc[0]-s2_df["UQL Adult Biomass"].iloc[0]
            

            diff_juv = s1_df["Juvenile Biomass"].iloc[0] - s2_df["Juvenile Biomass"].iloc[0]
            print(s1_df["Juvenile Biomass"].iloc[0], s2_df["Juvenile Biomass"].iloc[0])
            print(s1_df["Juvenile Biomass"].iloc[0] - s2_df["Juvenile Biomass"].iloc[0])
  
  
            diff_ad = s1_df["Adult Biomass"].iloc[0] - s2_df["Adult Biomass"].iloc[0]
            
            diff_biomass.append([spec,year,combi,diff_lql_juv,diff_uql_juv,diff_juv,diff_lql_ad,diff_uql_ad,diff_ad])
            
            
df = pd.DataFrame.from_records(diff_biomass, columns=["Species","Year","Seascape combination","Diff LQL Juv","Diff UQL Juv","Diff Juv Biomass","Diff LQL Adult","Diff UQL Adult","Diff Adult Biomass"])
df.to_csv(user+"RF/lag_combined_data_year_seascape_biomass_diff.csv")

#print(len(df))
#print(len(final))
#print(final.columns)
#print(df.columns)

#combine causality and diff in biomass data for seascape combinations
df_new=final.merge(df,how="outer")
df_new

#combine with trait/life-history table data to finalize training dataset for Random Forest analysis
trait_table

RF_table = df_new.merge(trait_table,how="outer")
RF_table

RF_table.to_csv(user+"RF/RF_table.csv")


[1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991
 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005
 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
Gadus morhua
1978
1.8725879890584043 3.7226322786769592
-1.850044289618555
13.842048678298006 12.55851241680766
1.2835362614903456
15.71463666735641 16.281144695484613
-0.5665080281282027
1.8725879890584043 2.1631643469595683
-0.290576357901164
13.842048678298006 3.6890202734441058
10.1530284048539
15.71463666735641 5.8521846204036745
9.862452046952736
1.8725879890584043 0.9575214597531084
0.9150665293052959
13.842048678298006 2.3294029803607272
11.512645697937279
15.71463666735641 3.2869244401138347
12.427712227242576
1.8725879890584043 2.2199112905592475
-0.34732330150084323
13.842048678298006 8.496420107020013
5.345628571277993
15.71463666735641 10.716331397579252
4.998305269777159
1.8725879890584043 9.336623910463947
-7.464035921405543
13.842048678298006 244.1561510901245

0.35081152134507765 0.2428855212291887
0.10792600011588896
14.095808489756232 1.249867391843635
12.845941097912597
14.446620011101306 1.4927529130728237
12.953867098028482
0.35081152134507765 12.261699901036724
-11.910888379691647
14.095808489756232 31.364425037951627
-17.268616548195396
14.446620011101306 43.62612493898836
-29.179504927887056
0.35081152134507765 0.26404134334899604
0.08677017799608161
14.095808489756232 0.7538598749571657
13.341948614799065
14.446620011101306 1.0179012183061618
13.428718792795145
0.35081152134507765 6.291445205539935
-5.940633684194857
14.095808489756232 21.920875165455996
-7.825066675699764
14.446620011101306 28.212320370995922
-13.765700359894616
2.546807916171031 1.426087577586569
1.120720338584462
14.713182962433136 17.996626128202838
-3.283443165769702
17.259990878604164 19.422713705789405
-2.1627228271852417
2.546807916171031 1.1687771105317843
1.3780308056392465
14.713182962433136 38.43686357352698
-23.72368061109384
17.259990878604164 39.60564

38.54701796576823 14.479048983901253
24.067968981866976
39.39884911205868 15.286574908551865
24.112274203506814
0.8518311462904512 0.7553449045259492
0.09648624176450205
38.54701796576823 10.117525232172797
28.42949273359543
39.39884911205868 10.872870136698744
28.525978975359934
0.8518311462904512 0.11568068163206435
0.7361504646583868
38.54701796576823 0.3300678962062015
38.216950069562024
39.39884911205868 0.4457485778382659
38.95310053422041
0.8518311462904512 3.4937259954264066
-2.6418948491359555
38.54701796576823 10.72867260492471
27.818345360843516
39.39884911205868 14.222398600351116
25.17645051170756
0.8518311462904512 3.0889075149034526
-2.2370763686130015
38.54701796576823 2.752434120756891
35.79458384501134
39.39884911205868 5.841341635660343
33.557507476398335
0.8518311462904512 6.8526834183286915
-6.00085227203824
38.54701796576823 30.90138202956057
7.645635936207658
39.39884911205868 37.75406544788925
1.6447836641694238
1.1022091968105854 1.6823248504784332
-0.580115653

1.0394929087646292 0.31532143167509064
0.7241714770895386
17.250811293353177 2.022474120584218
15.22833717276896
18.290304202117813 2.337795552259309
15.952508649858505
1.0394929087646292 8.349670917629059
-7.31017800886443
17.250811293353177 9.609947048132847
7.64086424522033
18.290304202117813 17.95961796576191
0.330686236355902
1.0394929087646292 0.22593805029319405
0.8135548584714352
17.250811293353177 1.3376540916710868
15.91315720168209
18.290304202117813 1.5635921419642809
16.72671206015353
1.0394929087646292 3.2537655230243243
-2.214272614259695
17.250811293353177 13.344019556780411
3.9067917365727656
18.290304202117813 16.59778507980473
1.692519122313083
2.9201608558437053 1.0587234042192752
1.86143745162443
13.991049482153885 7.913376861525219
6.077672620628666
16.911210337997588 8.97210026574449
7.939110072253097
2.9201608558437053 2.0502663495766242
0.8698945062670811
13.991049482153885 8.327033219103111
5.664016263050774
16.911210337997588 10.37729956867974
6.5339107693178

6.018733232939855
9.526825189319338 2.807913004982266
6.718912184337071
1.2346345169576407 0.32258615404185953
0.9120483629157812
8.292190672361697 16.785007976167687
-8.49281730380599
9.526825189319338 17.107594130209552
-7.580768940890215
0.5344555655604233 0.32258615404185953
0.21186941151856375
2.2734574394218416 16.785007976167687
-14.511550536745846
2.807913004982266 17.107594130209552
-14.299681125227286
1986
0.2706352236485648 0.5004543070353621
-0.2298190833867973
16.132023703644315 17.251538614297395
-1.1195149106530806
16.40265892729288 17.751992921332754
-1.349333994039874
0.2706352236485648 1.3282414687712052
-1.0576062451226405
16.132023703644315 13.520620972990287
2.611402730654028
16.40265892729288 14.848862441761492
1.5537964855313877
0.2706352236485648 1.0965128226371472
-0.8258775989885825
16.132023703644315 50.515939967132596
-34.38391626348828
16.40265892729288 51.612452789769755
-35.20979386247687
0.2706352236485648 1.8708892833604789
-1.6002540597119141
16.132023

...

0.0013134666418541305 0.0
0.0013134666418541305
0.0047318719958619575 0.0
0.0047318719958619575
0.006045338637716089 0.0
0.006045338637716089
0.0013134666418541305 0.002302990284891432
-0.0009895236430373015
0.0047318719958619575 0.0
0.0047318719958619575
0.006045338637716089 0.002302990284891432
0.003742348352824657
0.0013134666418541305 0.0
0.0013134666418541305
0.0047318719958619575 0.0
0.0047318719958619575
0.006045338637716089 0.0
0.006045338637716089
0.0013134666418541305 0.0017347348048039931
-0.0004212681629498626
0.0047318719958619575 0.00203853366700726
0.0026933383288546974
0.006045338637716089 0.003773268471811253
0.002272070165904836
0.0 0.002171223488990483
-0.002171223488990483
0.0 0.002314928585922658
-0.002314928585922658
0.0 0.00448615207491314
-0.00448615207491314
0.0 0.00012200803207959087
-0.00012200803207959087
0.0 0.0020868674370402053
-0.0020868674370402053
0.0 0.0022088754691197967
-0.0022088754691197967
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.001659544668624

0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.00037371155845072353
-0.00037371155845072353
0.0 0.000733141558624475
-0.000733141558624475
0.0 0.0011068531170751986
-0.0011068531170751986
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.00025249054005738213
-0.00025249054005738213
0.0 0.0
0.0
0.0 0.00025249054005738213
-0.00025249054005738213
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.00037371155845072353
-0.00037371155845072353
0.0 0.000733141558624475
-0.000733141558624475
0.0 0.0011068531170751986
-0.0011068531170751986
0.0 0.00025249054005738213
-0.00025249054005738213
0.0 0.0
0.0
0.0 0.00025249054005738213
-0.00025249054005738213
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.0
0.0
0.0 0.00037371155845072353
-0.00037371155845072353
0.0 0.000733141558624475
-0.000733141558624475
0.0 0.0011068531170751986
-0.0011068531170751986
0.00025249054005738213 0.0
0.00025249054005738213
0.0 0.0
0.0
0.00025249054005738213 0.0
0.00025249054005738213
0.00025249054005738213 0.00037371155845072353
-0.0001212210183933414
0.0 0.

,Seascape combination,Neighbouring,Species,Causal lag 0,Causal lag 1,Causal lag 2,Year,Diff LQL Juv,Diff UQL Juv,Diff Juv Biomass,Diff LQL Adult,Diff UQL Adult,Diff Adult Biomass
0,1-2,yes,Gadus morhua,no,no,no,1978,-1.850044,1.283536,-0.566508,0.434505,-66.391215,-65.956710
1,1-3,yes,Gadus morhua,no,no,no,1978,-0.290576,10.153028,9.862452,8.859824,3.755456,12.615280
2,1-4,no,Gadus morhua,no,no,no,1978,0.915067,11.512646,12.427712,8.236253,-7.848300,0.387953
3,1-5,no,Gadus morhua,no,no,no,1978,-0.347323,5.345629,4.998305,7.279339,0.173710,7.453049
4,1-6,no,Gadus morhua,no,no,no,1978,-7.464036,-230.314102,-237.778138,11.189940,6.073392,17.263332
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17005,7-9,yes,Callionymus lyra,no,no,no,2019,0.000000,0.000000,0.000000,-0.025934,-0.129221,-0.155155
17006,7-10,no,Callionymus lyra,yes,no,no,2019,-0.007489,-0.004483,-0.011972,-0.029029,-0.058969,-0.087999
17007,8-9,yes,Callionymus lyra,yes,yes,no,2019,0.000148,0.000000,0.000148,-0.007778,-0.127977,-0.135755
17008,8-10,yes,Callionymus lyra,no,no,no,2019,-0.007341,-0.004483,-0.011824,-0.010873,-0.057725,-0.068598


,Species,Family,Seasonal migration,Fecundity,Spawning type,Age at Maturity,Maximum age,Habitat,Feeding mode,Body type,Maximum length,Length at maturity
0,Gadus morhua,Gadidae,yes,4.836395e+06,non-guarder,4.858807,25.000000,benthopelagic,generalist,fusiform,116.258333,68.3
1,Pleuronectes platessa,Pleuronectidae,yes,5.520000e+05,non-guarder,4.500000,30.857143,demersal,benthivorous,flat,100.000000,30.8
2,Eutrigla gurnardus,Triglidae,yes,2.449490e+05,non-guarder,2.775000,2.775000,demersal,generalist,elongated,36.000000,18.8
3,Sprattus sprattus,Clupeidae,yes,1.463000e+04,non-guarder,2.000000,6.000000,pelagic,planktivorous,fusiform,14.440000,10.1
4,Limanda limanda,Pleuronectidae,yes,2.224286e+05,non-guarder,2.985714,12.142857,demersal,benthivorous,flat,43.428571,21.4
5,Clupea harengus,Clupeidae,yes,9.318433e+04,non-guarder,3.219762,18.090185,benthopelagic,planktivorous,fusiform,31.786667,16.7
6,Amblyraja radiata,Rajidae,limited,1.700000e+01,non-guarder,8.633077,27.000000,demersal,generalist,flat,105.308333,38.0
7,Merlangius merlangus,Gadidae,yes,3.521439e+05,non-guarder,1.948889,20.000000,benthopelagic,generalist,fusiform,45.900000,27.8
8,Callionymus lyra,Callionymidae,limited,5.000000e+03,non-guarder,2.500000,7.000000,demersal,benthivorous,elongated,30.000000,10.3


,Seascape combination,Neighbouring,Species,Causal lag 0,Causal lag 1,Causal lag 2,Year,Diff LQL Juv,Diff UQL Juv,Diff Juv Biomass,...,Seasonal migration,Fecundity,Spawning type,Age at Maturity,Maximum age,Habitat,Feeding mode,Body type,Maximum length,Length at maturity
0,1-2,yes,Gadus morhua,no,no,no,1978,-1.850044,1.283536,-0.566508,...,yes,4.836395e+06,non-guarder,4.858807,25.0,benthopelagic,generalist,fusiform,116.258333,68.3
1,1-3,yes,Gadus morhua,no,no,no,1978,-0.290576,10.153028,9.862452,...,yes,4.836395e+06,non-guarder,4.858807,25.0,benthopelagic,generalist,fusiform,116.258333,68.3
2,1-4,no,Gadus morhua,no,no,no,1978,0.915067,11.512646,12.427712,...,yes,4.836395e+06,non-guarder,4.858807,25.0,benthopelagic,generalist,fusiform,116.258333,68.3
3,1-5,no,Gadus morhua,no,no,no,1978,-0.347323,5.345629,4.998305,...,yes,4.836395e+06,non-guarder,4.858807,25.0,benthopelagic,generalist,fusiform,116.258333,68.3
4,1-6,no,Gadus morhua,no,no,no,1978,-7.464036,-230.314102,-237.778138,...,yes,4.836395e+06,non-guarder,4.858807,25.0,benthopelagic,generalist,fusiform,116.258333,68.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17005,7-9,yes,Callionymus lyra,no,no,no,2019,0.000000,0.000000,0.000000,...,limited,5.000000e+03,non-guarder,2.500000,7.0,demersal,benthivorous,elongated,30.000000,10.3
17006,7-10,no,Callionymus lyra,yes,no,no,2019,-0.007489,-0.004483,-0.011972,...,limited,5.000000e+03,non-guarder,2.500000,7.0,demersal,benthivorous,elongated,30.000000,10.3
17007,8-9,yes,Callionymus lyra,yes,yes,no,2019,0.000148,0.000000,0.000148,...,limited,5.000000e+03,non-guarder,2.500000,7.0,demersal,benthivorous,elongated,30.000000,10.3
17008,8-10,yes,Callionymus lyra,no,no,no,2019,-0.007341,-0.004483,-0.011824,...,limited,5.000000e+03,non-guarder,2.500000,7.0,demersal,benthivorous,elongated,30.000000,10.3
